Scrape the Wikipedia page, using Beautiful Soup package

Get url, decode page source

In [179]:
import requests
website_url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [180]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

Find source code containing table, extract table columns

In [181]:
My_table=soup.find('table',{'class':'wikitable sortable'})

In [182]:
rows=My_table.find_all('tr')
columns=[v.text.replace('\n','') for v in rows[0].find_all('th')]

Extract table values: 
1.Ignore rows with borough 'not assigned'
2.Set not assigned neighbourhood with the same value of borough
3.Sort the table by Postal code for convenience

In [183]:
import pandas as pd
df=pd.DataFrame(columns=columns)
for i in range(1,len(rows)):
    tds=rows[i].find_all('td')
    values=[v.text.replace('\n','') for v in tds]
    if values[1].lower()!='not assigned':
        if values[2].lower()=='not assigned':
            values[2]=values[1]
        df=df.append(pd.Series(values,index=columns),ignore_index=True)
df.sort_values(by='Postcode',inplace=True)
df.reset_index(drop=True,inplace=True)
#print(df)

Merge rows with same postal code, print the final table

In [184]:
df2=pd.DataFrame(columns=columns)
k=-1
i=0
while i+1<len(df):
    df2=df2.append(df.iloc[i])
    k+=1
    j=i+1
    while df.iloc[i]['Postcode']==df.iloc[j]['Postcode']:
        df2.iloc[k]['Neighbourhood']=df2.iloc[k]['Neighbourhood']+', '+df.iloc[j]['Neighbourhood']
        j+=1
    i=j    
print(df2)    

    Postcode      Borough                                      Neighbourhood
0        M1B  Scarborough                                     Rouge, Malvern
2        M1C  Scarborough             Port Union, Rouge Hill, Highland Creek
5        M1E  Scarborough                  Guildwood, Morningside, West Hill
8        M1G  Scarborough                                             Woburn
9        M1H  Scarborough                                          Cedarbrae
..       ...          ...                                                ...
193      M9M   North York                                   Humberlea, Emery
195      M9N         York                                             Weston
196      M9P    Etobicoke                                          Westmount
197      M9R    Etobicoke  Richview Gardens, Kingsview Village, St. Phill...
201      M9V    Etobicoke  Albion Gardens, Beaumond Heights, Humbergate, ...

[102 rows x 3 columns]


Print the shape of final table

In [185]:
print(df2.shape)

(102, 3)
